In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_05_ft'
    dataset_name = 'RNA_DatasetBaselineSplitbppV0'
    sn_train = True
    
    model_name = 'RNA_ModelV3'
    dim = 192
    depth = 12
    dim_head = 32
   
    epoch = 6
    lr = 5e-5
    wd = 0.05
    pct_start = 0.01
    md_wt = 'exp_05/models/model.pth'
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [4]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)

ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train,)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train', mask_only=True,sn_train=CFG.sn_train)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', sn_train=CFG.sn_train)
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', mask_only=True, sn_train=CFG.sn_train)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

16

In [ ]:
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less)],
                metrics=[MAE()])
learn.model.load_state_dict(torch.load(CFG.md_wt))
learn.to_fp16()
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

epoch,train_loss,valid_loss,mae,time
0,0.131944,0.154595,0.163189,26:09
1,0.125965,0.142299,0.151435,26:24
2,0.124138,0.139044,0.147942,26:27
3,0.122319,0.136731,0.145964,26:32
4,0.121740,0.135325,0.145396,26:34
5,0.120165,0.133501,0.143268,26:37
6,0.119799,0.132564,0.142500,26:35
7,0.119771,0.132544,0.142046,26:33
8,0.119457,0.132303,0.141104,26:47
9,0.118891,0.132546,0.141082,26:56


Better model found at epoch 0 with mae value: 0.16318869460175442.
Better model found at epoch 1 with mae value: 0.15143463661766338.
Better model found at epoch 2 with mae value: 0.14794208168545112.
Better model found at epoch 3 with mae value: 0.14596357918959324.
Better model found at epoch 4 with mae value: 0.14539597281718727.
Better model found at epoch 5 with mae value: 0.1432681756809649.
Better model found at epoch 6 with mae value: 0.1425002828984979.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 7 with mae value: 0.1420456001133798.
Better model found at epoch 8 with mae value: 0.14110379024610442.
Better model found at epoch 9 with mae value: 0.14108248349761304.
Better model found at epoch 10 with mae value: 0.13992253907970947.
Better model found at epoch 11 with mae value: 0.13956964655667162.
Better model found at epoch 12 with mae value: 0.13843895789508256.
Better model found at epoch 14 with mae value: 0.13794076983174416.
Better model found at epoch 15 with mae value: 0.1378850090625583.
Better model found at epoch 17 with mae value: 0.13744751666963442.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

Better model found at epoch 31 with mae value: 0.13483122836750067.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 32 with mae value: 0.13460958213260832.
Better model found at epoch 34 with mae value: 0.13449255045188208.
Better model found at epoch 36 with mae value: 0.1334137091431888.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Better model found at epoch 40 with mae value: 0.13297329692600757.
Better model found at epoch 41 with mae value: 0.13279533241041191.
Better model found at epoch 43 with mae value: 0.1323455567614857.
Better model found at epoch 44 with mae value: 0.13194375386767057.
Better model found at epoch 45 with mae value: 0.131857992406369.
Better model found at epoch 48 with mae value: 0.13177694688335723.
Better model found at epoch 49 with mae value: 0.1315447436156995.
Better model found at epoch 51 with mae value: 0.1312032118746505.
Better model found at epoch 53 with mae value: 0.1310143991087333.
Better model found at epoch 55 with mae value: 0.1308694162519128.
Better model found at epoch 56 with mae value: 0.13083116717225893.
Better model found at epoch 57 with mae value: 0.1308140259132624.
Better model found at epoch 58 with mae value: 0.13076736461155203.
Better model found at epoch 59 with mae value: 0.13073964599313298.
Better model found at epoch 62 with mae value: 0.1306572

In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)().cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")